## Processing of search results ##

In [7]:
import json
import ipywidgets as widgets
from IPython.display import display

class JSONEditorJupyter:
    def __init__(self):
        self.filename_widget = widgets.Text(description="Filename:", layout=widgets.Layout(width='50%'))
        self.load_button = widgets.Button(description="Load")
        self.save_button = widgets.Button(description="Save Data")
        self.key_text_area = widgets.Textarea(description="Current Key:", layout=widgets.Layout(height='300px', width='95%'))
        self.prev_button = widgets.Button(description="< Previous")
        self.next_button = widgets.Button(description="Next >")
        self.data_grid = widgets.VBox()
        self.validation_input = widgets.Text(description="Validation:")
        self.human_class_input = widgets.Text(description="Human Class:")
        self.apply_button = widgets.Button(description="Apply Changes") # Keep the button for explicit apply if needed
        self.apply_button.on_click(self.apply_grid_changes) # Keep the handler

        self.data = {}
        self.keys_list = []
        self.current_key_index = 0
        self.selected_row_index = None

        self.load_button.on_click(self.load_data)
        self.save_button.on_click(self.save_data)
        self.prev_button.on_click(self.prev_key)
        self.next_button.on_click(self.next_key)

        self.key_text_area.disabled = True

        navigation_buttons = widgets.HBox([self.prev_button, self.next_button])
        top_bar = widgets.HBox([self.filename_widget, self.load_button])
        layout = widgets.VBox([top_bar, self.key_text_area, navigation_buttons, self.data_grid, self.validation_input, self.human_class_input, self.apply_button, self.save_button])
        display(layout)

    def load_data(self, b):
        filename = self.filename_widget.value
        try:
            with open(filename, 'r') as f:
                self.data = json.load(f)
            self.keys_list = list(self.data.keys())
            self.current_key_index = 0
            self.update_key_display()
            self.update_grid()
            print("Data loaded successfully.")
        except FileNotFoundError:
            print(f"Error: File not found at '{filename}'")
        except json.JSONDecodeError:
            print("Error: Invalid JSON format.")

    def save_data(self, b):
        self.save_current_changes()
        filename = self.filename_widget.value
        try:
            with open(filename, 'w') as f:
                json.dump(self.data, f, indent=4)
            print("Data saved successfully.")
        except Exception as e:
            print(f"Error saving data: {e}")

    def update_key_display(self):
        if self.keys_list:
            current_key = self.keys_list[self.current_key_index]
            self.key_text_area.value = current_key
        else:
            self.key_text_area.value = ""

    def update_grid(self):
        self.data_grid.children = []
        if self.keys_list:
            current_key = self.keys_list[self.current_key_index]
            values = self.data.get(current_key, [])
            grid_rows = []
            for index, item in enumerate(values):
                original_data = item.get('original_data', {})
                item_id = original_data.get('id', '')
                extracted_text = original_data.get('extracted_text', '')
                score = item.get('score', '')
                validation_value = item.get('validation', '') or ''
                human_class_value = item.get('human_class', '') or ''

                id_label = widgets.Label(f"ID: {item_id}")
                score_label = widgets.Label(f"Score: {score}")
                extracted_text_area = widgets.Textarea(value=extracted_text, layout=widgets.Layout(height='100px', width='90%'))
                validation_input = widgets.Text(value=validation_value, placeholder='Enter validation')
                human_class_input = widgets.Text(value=human_class_value, placeholder='Enter human class')

                row_widgets = widgets.HBox([id_label, score_label, extracted_text_area, validation_input, human_class_input])
                grid_rows.append(row_widgets)

                # Store a reference to the input widgets and the index for updating
                row_widgets.index = index
                row_widgets.validation_input = validation_input
                row_widgets.human_class_input = human_class_input

            self.data_grid.children = grid_rows

    def save_current_changes(self):
        if self.keys_list:
            current_key = self.keys_list[self.current_key_index]
            values = self.data.get(current_key, [])
            for child in self.data_grid.children:
                if hasattr(child, 'index') and hasattr(child, 'validation_input') and hasattr(child, 'human_class_input'):
                    index = child.index
                    validation = child.validation_input.value
                    human_class = child.human_class_input.value
                    if 0 <= index < len(values):
                        values[index]['validation'] = validation
                        values[index]['human_class'] = human_class
            self.data[current_key] = values

    def prev_key(self, b):
        self.save_current_changes()
        if self.keys_list:
            self.current_key_index = (self.current_key_index - 1) % len(self.keys_list)
            self.update_key_display()
            self.update_grid()

    def next_key(self, b):
        self.save_current_changes()
        if self.keys_list:
            self.current_key_index = (self.current_key_index + 1) % len(self.keys_list)
            self.update_key_display()
            self.update_grid()

    def apply_grid_changes(self, b):
        self.save_current_changes()
        print("Changes applied.")

if __name__ == "__main__":
    editor = JSONEditorJupyter()